# Model MLCLSP

(ang. Multi-level Capacitated Lot-sizing Problem, MLCLSP)

### Parametry:

$\mathcal{T}$ -- zbiór okresów czasu (zmian, dni lub tygodni),

$\mathcal{N}$ -- zbiór wyrobów,

$\mathcal{M}$ -- zbiór maszyn,

$\mathcal{\phi(m)}$ -- zbiór wyrobów, które są produkowane na maszynie $m$

$a_i$ -- liczba wyrobów $i$ potrzebnych do wyprodukowania jednego elementu $j$,

$p_{i}$ -- czas potrzebny na wyprodukowanie wyrobu $i$,

$c_{ij}$ -- koszt przezbrojenia z produkcji wyrobu $i$ na produkcję ryobu $j$, $c_{ii} = 0$,

$E_{it}$ -- zewnętrzne zapotrzebowanie na wyrób $i$ w okresie $t$,

$I_{i0}$ -- zapas początkowy wyrobu $i$,

$h_i$ -- jednostkowy koszt utrzymywania zapasów wyrobu $i$,

$L_{mt}$ -- dostępny czas pracy maszyny $m$ w okresie $t$,

$s_{ij}$ -- czas potrzebny na przezbrojenie się z produkcji wyrobu $i$ na produkcję wyrobu $j$, $s_{ii} = 0$,


### Zmienne:

$I_{it}$ -- zapas wyrobu $i$ na koniec okresu $t$,

$x_{it}$ -- wielkość produkcji wyrobu $i$ w okresie $t$,

$\mu^{s}_{it}$ -- czas rozpoczęcia produkcji wyrobu $i$ w okresie $t$,

$\alpha_{itm} = 1$, jeżeli maszyna $m$ jest przezbrojona na wyrób $i$ na początku okresu $t$, $0$ w przeciwnym razie,

$T_{ijtm} = 1$, jeżeli nastąpiło przezbrojenie z wyrobu $i$ na wyrób $j$ na maszynie $m$ w okresie $t$, $0$ w przeciwnym razie,


In [845]:
import numpy as np
import pandas as pd

---
## Zaczytanie danych wejściowych z pliku excel

In [846]:
# nazwy kolejnych wyrobów, maszyn i okresów w pliku excel muszą być kolejnymi liczbami naturalnymi zaczynającymi się o cyfry 0
wejscie_wyroby = pd.read_excel('szablony/MLCLSP_STRUMIEN_wejscie.xlsx', sheet_name='WYROBY')
wejscie_okresy = pd.read_excel('szablony/MLCLSP_STRUMIEN_wejscie.xlsx', sheet_name='ZAPOTRZEBOWANIE')
wejscie_czas_produkcji = pd.read_excel('szablony/MLCLSP_STRUMIEN_wejscie.xlsx', sheet_name='CZAS_PRODUKCJI')
wejscie_czas_przezbrojenia = pd.read_excel('szablony/MLCLSP_STRUMIEN_wejscie.xlsx', sheet_name='CZAS_PRZEZBROJENIA')
wejscie_dostepnosc_maszyn = pd.read_excel('szablony/MLCLSP_STRUMIEN_wejscie.xlsx', sheet_name='DOSTEPNOSC_MASZYN')
wejscie_koszt_przezbrojenia = pd.read_excel('szablony/MLCLSP_STRUMIEN_wejscie.xlsx', sheet_name='KOSZT_PRZEZBROJENIA')
bom = pd.read_excel('szablony/MLCLSP_STRUMIEN_wejscie.xlsx', sheet_name='BOM')

print(wejscie_wyroby)
print(wejscie_okresy)
print(wejscie_czas_produkcji)
print(wejscie_czas_przezbrojenia)
print(wejscie_dostepnosc_maszyn)
print(wejscie_koszt_przezbrojenia)
print(bom)

   wyrob  hi  Ii0
0      0   3    0
1      1   2    0
2      2   2    0
3      3   1    0
   wyrob/okres  1  2
0            0  3  0
1            1  0  2
2            2  0  0
3            3  0  0
   pmi    0    1    2
0    0  0.1  0.0  0.0
1    1  0.0  0.4  0.0
2    2  0.0  0.0  0.1
3    3  0.0  0.0  0.1
   sij  0  1     2     3
0    0  0  0  0.00  0.00
1    1  0  0  0.00  0.00
2    2  0  0  0.00  0.05
3    3  0  0  0.05  0.00
   Lmt  1  2
0    0  1  1
1    1  1  1
2    2  1  1
   cij  0  1  2  3
0    0  0  0  0  0
1    1  0  0  0  0
2    2  0  0  0  5
3    3  0  0  5  0
   j/i  0  1  2  3
0    0  0  0  0  0
1    1  0  0  0  0
2    2  1  0  0  0
3    3  0  1  1  0


---
## Wyciągnięcie danych z zaczytanych tabel

In [847]:
zbior_wyrobow = wejscie_wyroby.loc[:, 'wyrob'].values
zbior_okresow = wejscie_okresy.iloc[:, 1:].columns.values
zbior_maszyn = wejscie_czas_produkcji.iloc[:, 1:].columns.values
zbior_okresow_wraz_z_poczatkowym = np.append(0, zbior_okresow)
zbior_okresow_dla_przezbrojen_poczatkowych = np.append(zbior_okresow,
                                                       zbior_okresow[-1] + 1)

print('Zbiór wyrobów: ', zbior_wyrobow)
print('Zbiór okresów: ', zbior_okresow)
print('Zbiór maszyn: ', zbior_maszyn)
print('Zbiór okresów z okresem początkowym: ', zbior_okresow_wraz_z_poczatkowym)
print('Zbiór okresów dla przezbrojen poczatkowych: ', zbior_okresow_dla_przezbrojen_poczatkowych)

Zbiór wyrobów:  [0 1 2 3]
Zbiór okresów:  [1 2]
Zbiór maszyn:  [0 1 2]
Zbiór okresów z okresem początkowym:  [0 1 2]
Zbiór okresów dla przezbrojen poczatkowych:  [1 2 3]


In [848]:
h = wejscie_wyroby.loc[:, 'hi'].values
I_0 = wejscie_wyroby.loc[:, 'Ii0'].values

print('Jednostkowy koszt utrzymania wyrobu i: ', h)
print('Zapasy początkowe: ', I_0)

Jednostkowy koszt utrzymania wyrobu i:  [3 2 2 1]
Zapasy początkowe:  [0 0 0 0]


#### Przypisanie wyrobów do odpowiednich maszyn

In [849]:
p = []
wyroby_przypisane_do_maszyn = []
zbior_maszyn_z_przypisanymi_wyrobami = []
pmi_tabela = wejscie_czas_produkcji.set_index('pmi').values

for wiersz in pmi_tabela:
    if np.count_nonzero(wiersz) == 1:
        machine_index = int(np.nonzero(wiersz)[0])
        p.append(wiersz[machine_index].item())
        wyroby_przypisane_do_maszyn.append(machine_index)
    else:
        raise Exception(
            "Każdy wyrób może być produkowany tylko na jednej maszynie. Należy poprawić dane w arkuszu 'CZAS_PRODUKCJI'. ")

wyroby_przypisane_do_maszyn = np.array(wyroby_przypisane_do_maszyn)
for i in zbior_maszyn:
    przypisane_wyroby = np.where(wyroby_przypisane_do_maszyn == i)[0].tolist()
    zbior_maszyn_z_przypisanymi_wyrobami.append(przypisane_wyroby)

print('Czas produkcji wyrobu i: ', p)
print('Wyroby przypisane do maszyn: ', wyroby_przypisane_do_maszyn)
print('Zbiór maszyn z przypisanymi wyrobami: ', zbior_maszyn_z_przypisanymi_wyrobami)

Czas produkcji wyrobu i:  [0.1, 0.4, 0.1, 0.1]
Wyroby przypisane do maszyn:  [0 1 2 2]
Zbiór maszyn z przypisanymi wyrobami:  [[0], [1], [2, 3]]


In [850]:
a = bom.set_index('j/i').values
E = wejscie_okresy.values
s = wejscie_czas_przezbrojenia.set_index('sij').values
L = wejscie_dostepnosc_maszyn.values
c = wejscie_koszt_przezbrojenia.set_index('cij').values

print('Macierz powiązań między produktami: \n', a)
print('Macierz z zewnętrznym zapotrzebowaniem na produkt i w okresie t (pierwsza kolumna to odpowiednik produktu): \n',
      E)
print('Czas potrzebny na przezbrojenie się z produkcji wyrobu i na produkcję ryobu j,: \n', s)
print('Koszt przezbrojenia z wyrobu i na wyrób j: \n', c)
print('Dostępność maszyny m w okresie t (pierwsza kolumna to odpowiednik maszyny): \n', L)

Macierz powiązań między produktami: 
 [[0 0 0 0]
 [0 0 0 0]
 [1 0 0 0]
 [0 1 1 0]]
Macierz z zewnętrznym zapotrzebowaniem na produkt i w okresie t (pierwsza kolumna to odpowiednik produktu): 
 [[0 3 0]
 [1 0 2]
 [2 0 0]
 [3 0 0]]
Czas potrzebny na przezbrojenie się z produkcji wyrobu i na produkcję ryobu j,: 
 [[0.   0.   0.   0.  ]
 [0.   0.   0.   0.  ]
 [0.   0.   0.   0.05]
 [0.   0.   0.05 0.  ]]
Koszt przezbrojenia z wyrobu i na wyrób j: 
 [[0 0 0 0]
 [0 0 0 0]
 [0 0 0 5]
 [0 0 5 0]]
Dostępność maszyny m w okresie t (pierwsza kolumna to odpowiednik maszyny): 
 [[0 1 1]
 [1 1 1]
 [2 1 1]]


---
## Sprawdzenie poprawności wpisanych danych dla czasu przezbrojenia oraz kosztów przezbrojenia

### Funkcje pomocnicze

In [851]:
def sprawdz_poprawnosc_dla(tabela, nazwa_arkusza):
    for kolumna in range(len(tabela[0])):
        for wiersz in range(len(tabela)):
            if tabela[wiersz, kolumna] != 0:
                if wiersz == kolumna:
                    raise Exception(
                        'Wartości [i,i] muszą być równe 0. Należy poprawić dane w arkuszu ' + nazwa_arkusza + ' dla: ' +
                        '(' + str(wiersz) + ',' + str(kolumna) + ')')
                else:
                    is_valid = False
                    for maszyna in zbior_maszyn_z_przypisanymi_wyrobami:
                        if wiersz in maszyna and kolumna in maszyna:
                            is_valid = True
                    if not is_valid:
                        raise Exception(
                            'Istnieją dane dla przezbrojeń, które są niemożliwe do wykonania. Należy poprawić dane w arkuszu ' + nazwa_arkusza + ' dla: ' +
                            '(' + str(wiersz) + ',' + str(kolumna) + ')')

In [852]:
sprawdz_poprawnosc_dla(s, 'CZAS_PRZEZBROJENIA')
sprawdz_poprawnosc_dla(c, 'KOSZT_PRZEZBROJENIA')

---
## Utworzenie modelu Gurobi oraz zmiennych

$I_{it} >= 0, x_{it} >= 0,  \mu^{s}_{it} >= 0
\alpha_{itm}, T_{ijtm} \in {0,1} \qquad i,j \in \mathcal{N}, t \in \mathcal{T}, m \in \mathcal{M}, $


In [853]:
import gurobipy as gb

model = gb.Model('MLCLSP_STRUMIEN')

# Deklaracje zmiennych (decyzyjnych)
x = {}
alfa = {}
T = {}
u = {}
I = {}

for i in zbior_wyrobow:
    for j in zbior_wyrobow:
        for t in zbior_okresow:
            for m in zbior_maszyn:
                T[i, j, t, m] = model.addVar(vtype=gb.GRB.BINARY,
                                             name='T(' + str(i) + ',' + str(j) + ',' + str(t) + ',' + str(m) + ')')
    for t in zbior_okresow:
        x[i, t] = model.addVar(vtype=gb.GRB.CONTINUOUS, lb=0, name='x(' + str(i) + ',' + str(t) + ')')
        u[i, t] = model.addVar(vtype=gb.GRB.CONTINUOUS, lb=0, name='u(' + str(i) + ',' + str(t) + ')')

print('Nazwa zmiennej:', 'x(' + str(1) + ',' + str(3) + ')')
for i in zbior_wyrobow:
    for t in zbior_okresow_wraz_z_poczatkowym:
        I[i, t] = model.addVar(vtype=gb.GRB.CONTINUOUS, lb=0, name='I(' + str(i) + ',' + str(t) + ')')
    for t in zbior_okresow_dla_przezbrojen_poczatkowych:
        for m in zbior_maszyn:
            alfa[i, t, m] = model.addVar(vtype=gb.GRB.BINARY, name='alfa(' + str(i) + ',' + str(t) + ',' + str(m) + ')')


Nazwa zmiennej: x(1,3)


---
## Funkcja celu
$\min \displaystyle\sum_{i \in \mathcal{N}} \sum_{t \in \mathcal{T}}h_i I_{it} + \displaystyle\sum_{i \in \mathcal{N}} \sum_{j \in \mathcal{N}} \sum_{t \in \mathcal{T}} \sum_{m \in \mathcal{M}}c_{ij} T_{ijtm}$

In [854]:
model.setObjective(gb.quicksum(h[i] * I[i, t] for i in zbior_wyrobow for t in zbior_okresow) +
                   gb.quicksum(
                       c[i, j] * T[i, j, t, m] for i in zbior_wyrobow for j in zbior_wyrobow for t in zbior_okresow for
                       m in zbior_maszyn))

---
## Ograniczenia
$I_{i0} = I^0_i$

In [855]:
model.addConstrs((I[i, 0] == I_0[i] for i in zbior_wyrobow), name='zapas-poczatkowy')
model.update()

$T_{ijtm} = 0 \qquad i,j \notin \mathcal{\phi(m)}, $\
$T_{ijtm} = 0 \qquad i=j, $\
$\alpha_{itm} = 0 \qquad i \notin \mathcal{\phi(m)}$

In [856]:
for m in zbior_maszyn:
    for t in zbior_okresow:
        for i in zbior_wyrobow:
            for j in zbior_wyrobow:
                if i==j or i not in zbior_maszyn_z_przypisanymi_wyrobami[m] or j not in zbior_maszyn_z_przypisanymi_wyrobami[m]:
                    model.addConstr(T[i,j,t,m] == 0, name='t_limit[i,j,t,m](' + str(i) + ','+ str(j) + ','+ str(t) + ',' + str(m) + ')')
            if i not in zbior_maszyn_z_przypisanymi_wyrobami[m]:
                model.addConstr(alfa[i,t,m] == 0,name='alfa_limit[i,t,m](' + str(i) + ',' + str(t) + ',' + str(m) + ')')

$I_{it} - I_{i(t-1)} - x_{it} + \sum_{j \in \mathcal{N}}( a_{ij} * x_{jt} ) + E_{it} = 0 \qquad i \in \mathcal{N}, t \in \mathcal{T},$

In [857]:
for i in zbior_wyrobow:
    for t in zbior_okresow:
        model.addConstr(
            I[i, t] - I[i, t - 1] - x[i, t] + gb.quicksum(a[i, j] * x[j, t] for j in zbior_wyrobow) + E[i, t] == 0,
            name='bilans_zapasow[i,t](' + str(i) + ',' + str(t) + ')'
        )

$  p_{i} * x_{it} <= \sum_{j \in \mathcal{\phi(m)}}(T_{jitm}) + \alpha_{itm} \qquad m \in \mathcal{M}, t \in \mathcal{T}, i \in \mathcal{\phi(m)},$

Wyrób jest produkowany tylko w przypadku, gdy maszyna jest na niego przezbrojona.

In [858]:
for m in zbior_maszyn:
    for t in zbior_okresow:
        for i in zbior_maszyn_z_przypisanymi_wyrobami[m]:
            model.addConstr(
                p[i] * x[i, t] <= gb.quicksum(T[j, i, t, m] for j in zbior_maszyn_z_przypisanymi_wyrobami[m]) + alfa[i, t, m],
                name='produkcja_gdy_przezbrojona[m,i,t](' + str(m) + ',' + str(i) + ',' + str(t) + ')'
            )

$ \sum_{i \in \mathcal{\phi(m)}}\alpha_{itm} = 1 \qquad m \in \mathcal{M}, t \in \mathcal{T},$

W danym okresie maszyna powinna być wstępnie przezbrojona tylko na 1 wyrób.

In [859]:
for m in zbior_maszyn:
    for t in zbior_okresow:
        model.addConstr(gb.quicksum(alfa[i, t, m] for i in zbior_maszyn_z_przypisanymi_wyrobami[m]) == 1,
                        name='przezbrojenie_wstepne_limit[m,t](' + str(m) + ',' + str(t) + ')'
                        )

$ \sum_{j \in \mathcal{\phi(m)}}(T_{jitm}) + \alpha_{itm} = \sum_{j \in \mathcal{\phi(m)}}(T_{ijtm}) + \alpha_{i(t+1)m}
 \qquad m \in \mathcal{M}, t \in \mathcal{T}, i \in \mathcal{\phi(m)},$

Jeżeli maszyna jest przezbrojona na dany wyrób i w okresie t, to musi nastąpić zmiana na inny wyrób w tym samym okresie lub stan pozostanie taki jaki jest tzn. maszyna nadal będzie przezbrojona na wyrób i w okresie t+1.

In [860]:
for m in zbior_maszyn:
    for t in zbior_okresow:
        for i in zbior_maszyn_z_przypisanymi_wyrobami[m]:
            model.addConstr(gb.quicksum(T[j, i, t, m] for j in zbior_maszyn_z_przypisanymi_wyrobami[m]) + alfa[i, t, m] == gb.quicksum(
                T[i, j, t, m] for j in zbior_maszyn_z_przypisanymi_wyrobami[m]) + alfa[i, t + 1, m],
                            name='przezbrojenie_zapewnienie_stanu[m,i,t](' + str(m) + ',' + str(i) + ',' + str(t) + ')'
                            )

$ \sum_{j \in \mathcal{\phi(m)}}T_{jitm} <= 1  \qquad m \in \mathcal{M}, t \in \mathcal{T}, i \in \mathcal{\phi(m)},$

Nie można przezbroić maszyny dwa razy na ten sam produkt w tym samym okresie.

In [861]:
for m in zbior_maszyn:
    for t in zbior_okresow:
        for i in zbior_maszyn_z_przypisanymi_wyrobami[m]:
            model.addConstr(gb.quicksum(T[j, i, t, m] for j in zbior_maszyn_z_przypisanymi_wyrobami[m]) <= 1,
                            name='przezbrojenie_raz_na_okres[m,i,t](' + str(m) + ',' + str(i) + ',' + str(t) + ')'
                            )

$ \mu^{s}_{it} + p_i * x_{it} + s{ij} * T_{ijtm} + T_{ijtm} - 1 - \alpha_{j(t+1)m} <= \mu^{s}_{jt}
\qquad m \in \mathcal{M}, t \in \mathcal{T}, i \in \mathcal{\phi(m)}, j \in \mathcal{\phi(m)}, $ \

$ \mu^{s}_{it} + p_i * x_{it} + s{ij} * T_{ijtm} + \alpha_{j(t+1)m} - 1 - \alpha_{jtm} <= \mu^{s}_{jt}
\qquad m \in \mathcal{M}, t \in \mathcal{T}, i \in \mathcal{\phi(m)}, j \in \mathcal{\phi(m)}, $

Produkcja i przebrojenia muszą zostać tak dobrane, aby czasy przezbrojeń i startu produkcji wyrobów się zgadzały.
Drugi warunek pozwala na określenie czasu startu produkcji ostatniego wyrobu produkowanego w okresie t na maszynie m.

In [862]:
for m in zbior_maszyn:
    for t in zbior_okresow:
        for i in zbior_maszyn_z_przypisanymi_wyrobami[m]:
            for j in zbior_maszyn_z_przypisanymi_wyrobami[m]:
                if i != j:
                    model.addConstr(
                        u[i, t] + p[i] * x[i, t] + s[i, j] * T[i, j, t, m] + T[i, j, t, m] - 1 - alfa[j, t + 1, m] <= u[
                            j, t],
                        name='start_produkcji_wyrobu[i,j,m,t](' + str(i) + ',' + str(j) + ',' + str(m) + ',' + str(t) + ')'
                    )
                    model.addConstr(
                        u[i, t] + p[i] * x[i, t] + s[i, j] * T[i, j, t, m] + alfa[j, t + 1, m] - 1 - alfa[j, t, m] <= u[
                            j, t],
                        name='start_produkcji_ostatniego_wyrobu[i,j,m,t](' + str(i) + ',' + str(j) + ',' + str(
                            m) + ',' + str(t) + ')'
                    )

$ \mu^{s}_{it} + p_i * x_{it} + \sum_{j \in \mathcal{\phi(m)}} s_{ij} * T_{ijtm} <= 1
 \qquad m \in \mathcal{M}, t \in \mathcal{T}, i \in \mathcal{\phi(m)},$

Limit czasu pracy maszyny.

In [863]:
for m in zbior_maszyn:
    for t in zbior_okresow:
        for i in zbior_maszyn_z_przypisanymi_wyrobami[m]:
            model.addConstr(
                u[i, t] + p[i] * x[i, t] + gb.quicksum(
                    s[i, j] * T[i, j, t, m] for j in zbior_maszyn_z_przypisanymi_wyrobami[m]) <= L[m, t],
                name='limit_czasu_pracy_maszyny[m,t,i](' + str(m) + ',' + str(t) + ',' + str(i) + ')'
            )

$ I_{i(t-1)} >= \sum_{j \in \mathcal{i}} (a_{ij} * \min \displaystyle (x_{jt}, \frac{1}{p_{j}}(\mu^{s}_{it}-\mu^{s}_{jt} )^+) \qquad i \in \mathcal{N}, t \in \mathcal{T},$

Zapas wyrobu i w okresie t-1 musi zaspokoić początkowe zapotrzebowanie na wyrób i w okresie t dla wszystkich wyrobów, które składają się z wyrobu i. Jeżeli produkcja i będzie kontunuowana później niż czas rozpoczęcia produkcji wyrobu j to potrzebujemy zapasu początkowego, który pozwoli nam na produkcję wyrobu j.

In [864]:
# Dużo zmiennych pomocniczych bo w Gurobi w max i min nie może być operacji arytmetycznych a python max, min ma niezgodność typów

In [865]:
aux1 = {}
aux2 = {}
aux3 = {}
aux4 = {}
for i in zbior_wyrobow:
    for t in zbior_okresow:
        for j in zbior_wyrobow:
            aux1[i, j, t] = model.addVar(vtype=gb.GRB.CONTINUOUS, lb=-gb.GRB.INFINITY, name='aux1(' + str(i) + ',' + str(j) + ',' + str(t) + ')')
            aux2[i, j, t] = model.addVar(vtype=gb.GRB.CONTINUOUS, name='aux2(' + str(i) + ',' + str(j) + ',' + str(t) + ')')
            aux3[i, j, t] = model.addVar(vtype=gb.GRB.CONTINUOUS, name='aux3(' + str(i) + ',' + str(j) + ',' + str(t) + ')')
            aux4[i, j, t] = model.addVar(vtype=gb.GRB.CONTINUOUS, name='aux4(' + str(i) + ',' + str(j) + ',' + str(t) + ')')
for i in zbior_wyrobow:
    for t in zbior_okresow:
        for j in zbior_wyrobow:
            model.addConstr(aux1[i, j, t] == u[i, t] - u[j, t], name='aux1[i,j,t](' + str(i) + ',' + str(j) + ',' + str(t) + ')')
            model.addConstr(aux2[i, j, t] == gb.max_([0, aux1[i, j, t]]), name='aux2[i,j,t](' + str(i) + ',' + str(j) + ',' + str(t) + ')')
            model.addConstr(aux3[i, j, t] == aux2[i, j, t] * (1 / p[j]), name='aux3[i,j,t](' + str(i) + ',' + str(j) + ',' + str(t) + ')')
            model.addConstr(aux4[i, j, t] == gb.min_([x[j, t], aux3[i, j, t]]), name='aux4[i,j,t](' + str(i) + ',' + str(j) + ',' + str(t) + ')')


In [866]:
for i in zbior_wyrobow:
    for t in zbior_okresow:
        model.addConstr(
            I[i, t - 1] >= sum(a[i, j] * aux4[i, j, t] for j in zbior_wyrobow),
            name='zapewnienie_zapasu_poczatkowego[i,t](' + str(i) + ',' + str(t) + ')'
        )

$ I_{i(t-1)} + \min \displaystyle (x_{it}, \frac{1}{p_{i}}(\mu^{s}_{kt} + p_k*x_{kt} - \mu^{s}_{it})^+) >=
\sum_{j \in \mathcal{i}} (a_{ij} * \min \displaystyle (x_{jt}, \frac{1}{p_{j}}(\mu^{s}_{kt} + p_k*x_{kt} - \mu^{s}_{jt} )^+) \qquad i,k \in \mathcal{N}, t \in \mathcal{T},$

Zapas wyrobu i z okresu t-1 wraz z jego produkcją w okresie t musi zaspokoić zapotrzebowanie na wyrób i w okresie t.


In [867]:
aux5 = {}
aux6 = {}
aux7 = {}
aux8 = {}
for i in zbior_wyrobow:
    for t in zbior_okresow:
        for k in zbior_wyrobow:
            aux5[i, k, t] = model.addVar(vtype=gb.GRB.CONTINUOUS, lb=-gb.GRB.INFINITY, name='aux5(' + str(i) + ',' + str(k) + ',' + str(t) + ')')
            aux6[i, k, t] = model.addVar(vtype=gb.GRB.CONTINUOUS, name='aux6(' + str(i) + ',' + str(k) + ',' + str(t) + ')')
            aux7[i, k, t] = model.addVar(vtype=gb.GRB.CONTINUOUS, name='aux7(' + str(i) + ',' + str(k) + ',' + str(t) + ')')
            aux8[i, k, t] = model.addVar(vtype=gb.GRB.CONTINUOUS, name='aux8(' + str(i) + ',' + str(k) + ',' + str(t) + ')')
for i in zbior_wyrobow:
    for t in zbior_okresow:
        for k in zbior_wyrobow:
            model.addConstr(aux5[i, k, t] == u[k, t] + p[k] * x[k, t] - u[i, t], name='aux5[i,k,t](' + str(i) + ',' + str(k) + ',' + str(t) + ')')
            model.addConstr(aux6[i, k, t] == gb.max_([0, aux5[i, k, t]]), name='aux6[i,k,t](' + str(i) + ',' + str(k) + ',' + str(t) + ')')
            model.addConstr(aux7[i, k, t] == aux6[i, k, t] * (1 / p[i]), name='aux7[i,k,t](' + str(i) + ',' + str(k) + ',' + str(t) + ')')
            model.addConstr(aux8[i, k, t] == gb.min_([x[i, t], aux7[i, k, t]]), name='aux8[i,k,t](' + str(i) + ',' + str(k) + ',' + str(t) + ')')

In [868]:
for i in zbior_wyrobow:
    for k in zbior_wyrobow:
        for t in zbior_okresow:
            model.addConstr(
                I[i, t - 1] + aux8[i, k, t] >= sum(a[i, j] * aux8[j, k, t] for j in zbior_wyrobow),
                name='zapewnienie_zapasu[i,k,t](' + str(i) + ',' + str(k) + ',' + str(t) + ')'
            )

---
## Optymalizacja

In [869]:
# Aktualizacja całego modelu, obowiązkowa przed optymalizacją
model.update()

# Optymalizacja
model.optimize()

# Zapisanie modelu do pliku
model.write('szablony/MLCLSP_STRUMIEN_model.lp')
model.write('szablony/MLCLSP_STRUMIEN_model.mps')


Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 334 rows, 416 columns and 700 nonzeros
Model fingerprint: 0x3fe598c8
Model has 128 general constraints
Variable types: 284 continuous, 132 integer (132 binary)
Coefficient statistics:
  Matrix range     [5e-02, 1e+01]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 99 rows and 208 columns
Presolve time: 0.03s
Presolved: 235 rows, 208 columns, 702 nonzeros
Variable types: 143 continuous, 65 integer (65 binary)

Root relaxation: objective 6.000000e+00, 35 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    6.00000    0    5 

---
## Wydruk modelu

### Funkcje pomocnicze

In [871]:

def zamien_kropkw_na_przecinek(x):
    return (str(x).replace('.', ','))


def wypisz_parametry_modelu(writer):
    writer.writerow(['Zmienna', 'Wyrób i', 'Okres', 'Maszyna', 'Wyrób j', 'Wartość'])

    writer.writerow(['Nazwa modelu', '', '', '', '', model.ModelName])
    writer.writerow(['Wersja solvera', '', '', '', '', 'Gurobi {}.{}.{}'.format(*gb.gurobi.version())])
    writer.writerow(['Data rozwiązania', '', '', '', '', time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime())])
    writer.writerow(['Liczba ograniczeń', '', '', '', '', model.NumConstrs])
    writer.writerow(['Liczba zmiennych', '', '', '', '', model.NumVars])
    writer.writerow(['Liczba zmiennych binarnych', '', '', '', '', model.NumBinVars])
    writer.writerow(['Czar rozwiązywania', '', '', '', '', zamien_kropkw_na_przecinek(round(model.Runtime, 2))])
    writer.writerow(['Liczba iteracji', '', '', '', '', int(model.IterCount)])
    writer.writerow(['Rozwiązanie', '', '', '', '', zamien_kropkw_na_przecinek(round(model.ObjBound, 2))])

    writer.writerow(['n', '', '', '', '', zbior_wyrobow.size])
    writer.writerow(['T', '', '', '', '', zbior_okresow.size])

    # dane wejsciowe
    for wyrob_i in zbior_wyrobow:
        writer.writerow(['h', wyrob_i, '', '', '', zamien_kropkw_na_przecinek(h[wyrob_i])])
        writer.writerow(['I_0', '', '0', '', '', zamien_kropkw_na_przecinek(I_0[wyrob_i])])
        writer.writerow(['p', wyrob_i, '', '', '', zamien_kropkw_na_przecinek(p[wyrob_i])])

    for wyrob_i in zbior_wyrobow:
        for wyrob_j in zbior_wyrobow:
            writer.writerow(['a', wyrob_i, '', '', wyrob_j, zamien_kropkw_na_przecinek(a[wyrob_i, wyrob_j])])
            writer.writerow(['s', wyrob_i, '', '', wyrob_j, zamien_kropkw_na_przecinek(s[wyrob_i, wyrob_j])])
            writer.writerow(['c', wyrob_i, '', '', wyrob_j, zamien_kropkw_na_przecinek(c[wyrob_i, wyrob_j])])

    for wyrob_i in zbior_wyrobow:
        for okres in zbior_okresow:
            writer.writerow(['E', wyrob_i, okres, '', '', zamien_kropkw_na_przecinek(E[wyrob_i, okres])])

    for okres in zbior_okresow:
        for maszyna in zbior_maszyn:
            writer.writerow(['L', '', okres, maszyna, '', zamien_kropkw_na_przecinek(L[maszyna, okres])])

### Wydruk

In [872]:
import csv, time

plik_z_rozwiazaniem = 'szablony/MLCLSP_STRUMIEN_wyniki.csv'

with open(plik_z_rozwiazaniem, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=';', quoting=csv.QUOTE_NONE)
    # Czy rozwiazanie jest wykonalne
    if model.status == gb.GRB.INFEASIBLE:
        # rowiązanie niewykonalne
        model.computeIIS()
        wypisz_parametry_modelu(writer)
        writer.writerow(['Przez podane zmienne model jest niewykonalny:'])
        for c in model.getConstrs():
            if c.IISConstr:
                writer.writerow(['Parametr', c.constrName])
    else:
        # rozwiązanie wykonalne
        wypisz_parametry_modelu(writer)

        for wyrob_i in zbior_wyrobow:
            for okres in zbior_okresow:
                writer.writerow(
                    ['x', wyrob_i, okres, '', '', zamien_kropkw_na_przecinek(round(x[wyrob_i, okres].x, 2))])
                writer.writerow(
                    ['I', wyrob_i, okres, '', '', zamien_kropkw_na_przecinek(round(I[wyrob_i, okres].x, 2))])
                writer.writerow(
                    ['u', wyrob_i, okres, '', '', zamien_kropkw_na_przecinek(round(u[wyrob_i, okres].x, 2))])
                for wyrob_j in zbior_wyrobow:
                    for maszyna in zbior_maszyn:
                        writer.writerow(['T', wyrob_i, okres, maszyna, wyrob_j,
                                         zamien_kropkw_na_przecinek(round(T[wyrob_i, wyrob_j, okres, maszyna].x, 2))])
                for maszyna in zbior_maszyn:
                    writer.writerow(['alpha', wyrob_i, okres, maszyna, '',
                                     zamien_kropkw_na_przecinek(round(alfa[wyrob_i, okres, maszyna].x, 2))])
